# **Assignment 1 For Clustering Data Analytics**
<hr>

**Submitted by:** Jude M. Ando & Niles Vincent Gabrielle Rondez <br>
**Course & Year:** BSCS - 2 <br>
**Schedule:** M W 10:30 AM - 12:00 PM <br>
<hr>

## Instruction

@all 
Assignment 1 for Clustering:
New and novel methods in Machine Learning are made either by borrowing formulas and concepts from other scientific fields and redefining it based on new sets of assumptions, or by adding an extra step to an already existing framework of methodology.

In this exercise (Assignment 1 of the Clustering Topic), we will try to develop a novel method of Target Trial Emulation by integrating concepts of Clustering into the already existing framework. Target Trial Emulation is a new methodological framework in epidemiology which tries to account for the biases in old and traditional designs.

These are the instructions:
1. Look at this website: https://rpubs.com/alanyang0924/TTE
2. Extract the dummy data in the package and save it as "data_censored.csv"
2. Convert the R codes into Python Codes (use Jupyter Notebook), replicate the results using your python code.
3. Create another copy of your Python Codes, name it TTE-v2 (use Jupyter Notebook).
4. Using TTE-v2, think of a creative way on where you would integrate a clustering mechanism, understand each step carefully and decide at which step a clustering method can be implemented. Generate insights from your results.
5. Do this by pair, preferably your thesis partner.
6. Push to your github repository.
7. Deadline is 2 weeks from today: February 28, 2025 at 11:59 pm.

HINT: For those who dont have a thesis topic yet, you can actually develop a thesis topic out of this assignment.
<hr>

## Introduction


## 1. Setup

In [76]:
# 1. SETUP
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# Define directories for output (similar to tempdir() in R)
trial_pp_dir = "trial_pp"
trial_itt_dir = "trial_itt"

os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)


## 2. Data Preparation

In [77]:
# 2. DATA PREPARATION - Load dataset
data_censored = pd.read_csv("data_censored.csv")

# Ensure required columns exist
required_columns = ["id", "period", "treatment", "outcome", "eligible", "censored", "x1", "x2", "x3", "age"]
missing_columns = [col for col in required_columns if col not in data_censored.columns]

if missing_columns:
    raise KeyError(f"Missing columns in data_censored: {missing_columns}")

# Add necessary columns
data_censored["assigned_treatment"] = data_censored["treatment"]
data_censored["followup_time"] = data_censored["period"]

# Create trial objects as dictionaries
trial_pp = {"data": data_censored.copy()}
trial_itt = {"data": data_censored.copy()}

print(data_censored.head())  # Verify dataset is correctly loaded


   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   

   censored  eligible  assigned_treatment  followup_time  
0         0         1                   1              0  
1         0         0                   1              1  
2         0         0                   1              2  
3         0         0                   1              3  
4         0         0                   1              4  


## 3. Weight Models and Censoring

### 3.1 Censoring Due to Treatment Switching

In [78]:
# Logistic Regression for treatment switching
switch_model = LogisticRegression()
X_switch = data_censored[["age", "x1", "x3"]]
y_switch = data_censored["treatment"]

switch_model.fit(X_switch, y_switch)
data_censored["switch_weight"] = switch_model.predict_proba(X_switch)[:, 1]

### 3.2 Other Informative Censoring

In [81]:
# Logistic Regression for censoring
censor_model = LogisticRegression()
X_censor = data_censored[["x2", "x1"]]
y_censor = 1 - data_censored["censored"]

censor_model.fit(X_censor, y_censor)
data_censored["censor_weight"] = censor_model.predict_proba(X_censor)[:, 1]

## 4. Calculate Weights

In [93]:
# 4. CALCULATE WEIGHTS

# Compute final weight
data_censored["final_weight"] = data_censored["switch_weight"] * data_censored["censor_weight"]
data_censored["final_weight"] = data_censored["final_weight"].replace(0, 1).fillna(1)

# Merge weights into expanded dataset
trial_itt_expanded = trial_itt["data"].copy()
trial_itt_expanded = trial_itt_expanded.merge(
    data_censored[["id", "final_weight"]], on="id", how="left"
)

# Assign weight column safely
trial_itt_expanded["weight"] = trial_itt_expanded["final_weight"].fillna(1)
trial_itt_expanded.drop(columns=["final_weight"], inplace=True)

print(trial_itt_expanded.shape)  # Check size after merge


(10949, 15)


## 5. Specify Outcome Model

In [ ]:
# 5. SPECIFY OUTCOME MODEL
X_outcome = sm.add_constant(data_censored[["assigned_treatment", "x2", "followup_time"]])
y_outcome = data_censored["outcome"]

outcome_model = sm.Logit(y_outcome, X_outcome).fit()
print(outcome_model.summary())


Optimization terminated successfully.
         Current function value: 0.076672
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                outcome   No. Observations:                  725
Model:                          Logit   Df Residuals:                      721
Method:                           MLE   Df Model:                            3
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.02457
Time:                        19:43:11   Log-Likelihood:                -55.587
converged:                       True   LL-Null:                       -56.987
Covariance Type:            nonrobust   LLR p-value:                    0.4235
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -3.5845      0.520     -6.889      0.000      -4.604      -2.565
assig

## 6. Expand Trials

In [ ]:
# 6. EXPAND TRIALS
def create_sequence_of_trials(data, max_periods=3):
    expanded_trials = []
    
    for _, row in data.iterrows():
        period_start = int(row["period"])  
        period_end = min(period_start + max_periods, period_start + 10)

        for t in range(period_start, period_end):
            new_row = row.copy()
            new_row["trial_period"] = t
            new_row["followup_time"] = t - period_start
            expanded_trials.append(new_row)

    return pd.DataFrame(expanded_trials)

trial_itt_expanded = create_sequence_of_trials(trial_itt["data"])

print(trial_itt_expanded.head())  # Check if expansion works correctly


    id  period  treatment   x1        x2   x3        x4   age     age_s  \
0  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
0  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
0  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
1  1.0     1.0        1.0  1.0  0.002200  0.0  0.734203  37.0  0.166667   
1  1.0     1.0        1.0  1.0  0.002200  0.0  0.734203  37.0  0.166667   

   outcome  censored  eligible  assigned_treatment  followup_time  \
0      0.0       0.0       1.0                 1.0            0.0   
0      0.0       0.0       1.0                 1.0            1.0   
0      0.0       0.0       1.0                 1.0            2.0   
1      0.0       0.0       0.0                 1.0            0.0   
1      0.0       0.0       0.0                 1.0            1.0   

   trial_period  
0           0.0  
0           1.0  
0           2.0  
1           1.0  
1           2.0  


### 6.1 Create Sequence of Trials Data

In [103]:
# 6.1 RE-MERGE WEIGHT INTO EXPANDED TRIAL DATA
trial_itt_expanded = trial_itt_expanded.merge(
    data_censored[["id", "final_weight"]].rename(columns={"final_weight": "weight"}), 
    on="id", 
    how="left"
)

# Ensure weight column exists
trial_itt_expanded["weight"] = trial_itt_expanded["weight"].fillna(1)

print(trial_itt_expanded.head())  # ✅ THIS WILL NOW WORK FOR REAL


    id  period  treatment   x1        x2   x3        x4   age     age_s  \
0  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
1  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
2  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
3  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
4  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   

   outcome  censored  eligible  assigned_treatment  followup_time  \
0      0.0       0.0       1.0                 1.0            0.0   
1      0.0       0.0       1.0                 1.0            0.0   
2      0.0       0.0       1.0                 1.0            0.0   
3      0.0       0.0       1.0                 1.0            0.0   
4      0.0       0.0       1.0                 1.0            0.0   

   trial_period    weight  
0           0.0  0.579469  
1           0.0  0.591720  
2           0.0  0.506469  
3           0.0  0.486

## 7. Load or Sample From Expanded Data

In [104]:
# 7. LOAD OR SAMPLE FROM EXPANDED DATA
sample_size = min(int(len(trial_itt_expanded) * 0.5), len(trial_itt_expanded))

trial_itt_sampled = trial_itt_expanded[
    ["id", "assigned_treatment", "x2", "followup_time", "weight", "outcome"]
].sample(sample_size, random_state=1234).copy()

print(trial_itt_sampled.head())  # Verify columns exist


         id  assigned_treatment        x2  followup_time    weight  outcome
7937   23.0                 0.0  2.051176            0.0  0.229819      0.0
15279  46.0                 1.0  0.695822            2.0  0.553623      0.0
4515   16.0                 1.0  0.294230            2.0  0.663998      0.0
32182  98.0                 0.0  1.045454            1.0  0.208331      0.0
6071   21.0                 0.0 -0.805011            0.0  0.274840      0.0


## 8. Fit Marginal Structural Model

In [105]:
# 8. FIT MARGINAL STRUCTURAL MODEL

def winsorize_weights(weights):
    q99 = np.percentile(weights, 99)
    return np.minimum(weights, q99)

trial_itt_sampled["adjusted_weight"] = winsorize_weights(trial_itt_sampled["weight"])

X_msm = sm.add_constant(trial_itt_sampled[["assigned_treatment", "x2", "followup_time"]])
y_msm = trial_itt_sampled["outcome"]

msm_model = sm.Logit(y_msm, X_msm).fit()
print(msm_model.summary())


Optimization terminated successfully.
         Current function value: 0.040600
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                outcome   No. Observations:                16423
Model:                          Logit   Df Residuals:                    16419
Method:                           MLE   Df Model:                            3
Date:                Sun, 09 Mar 2025   Pseudo R-squ.:                 0.04070
Time:                        19:46:47   Log-Likelihood:                -666.77
converged:                       True   LL-Null:                       -695.06
Covariance Type:            nonrobust   LLR p-value:                 3.163e-12
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -4.4983      0.154    -29.118      0.000      -4.801      -4.195
assi

## 9. Inference

In [7]:
# 9. INFERENCE - Predict Survival Differences

predict_times = np.arange(0, 11)
new_data = trial_itt_sampled.copy()

# Ensure constant column exists
new_data = sm.add_constant(new_data[["assigned_treatment", "x2", "followup_time"]])

predictions = []
for t in predict_times:
    new_data["followup_time"] = t
    pred_prob = msm_model.predict(new_data)
    predictions.append(pred_prob.mean())

predictions = np.array(predictions)
std_error = np.std(predictions) / np.sqrt(len(predictions))
ci_lower = predictions - 1.96 * std_error
ci_upper = predictions + 1.96 * std_error

plt.plot(predict_times, predictions, label="Survival Difference", color="blue")
plt.fill_between(predict_times, ci_lower, ci_upper, color="red", alpha=0.3, label="95% CI")

plt.xlabel("Follow-up Time")
plt.ylabel("Survival Probability")
plt.legend()
plt.title("Survival Difference Over Time")

plt.show()


NameError: name 'np' is not defined